<a href="https://colab.research.google.com/github/ryandoyle5401/CMPSC472HW14/blob/add-lock/HW14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
%%writefile test.c

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <fcntl.h>
#include <sys/stat.h>
#include <sys/types.h>
#include <string.h>
#include <pthread.h>

#define BUFFER_SIZE 1024


//Note: the create_file, read_file are from the CMPSC472_11 colab.

// Used for creating and writing to a file. Probably create a function for writing to a file.
int create_file(const char *filename) {
    int fd = open(filename, O_CREAT | O_RDWR | O_APPEND);
    if (fd == -1) {
        perror("open");
        exit(EXIT_FAILURE);
    }

    printf("File '%s' created.\n", filename);
    return fd;
}

// Used for writing to a file.
void write_file(int fd, const char *data) {
    if (write(fd, data, 19) == -1) {
        perror("write");
        close(fd);
        exit(EXIT_FAILURE);
    }
}

// Used for reading from a file.
void read_file(int fd) {
    char buffer[BUFFER_SIZE];
    ssize_t bytes_read;

    // Read data from the file
    bytes_read = read(fd, buffer, BUFFER_SIZE-1);
    if (bytes_read == -1) {
        perror("read");
        close(fd);
        exit(EXIT_FAILURE);
    }

    // Null-terminate the buffer to print as string
    buffer[bytes_read] = '\0';

    printf("Read from file: %s\n", buffer);
}

// Used to check lock status
void check_lock(int fd, struct flock *fl) {
    // Check the lock status using the same `fl` struct
    if (fcntl(fd ,F_GETLK, fl) == 0 && fl->l_type != F_UNLCK) {
        printf("Child: File is locked by another process (PID: %d), cannot write.\n", fl->l_pid);
    } else {
        printf("Child: Lock is free, writing to the file.\n");
        write_file(fd, "Child writing to file.\n");
    }
}

// Used to release the lock
void release_lock(int fd, struct flock *fl) {
    // Release the lock
    fl->l_type = F_UNLCK;
    fcntl(fd, F_SETLK, fl);
}

int main() {
    const char *filename = "testfile.txt";

    // Create a file and write to it
    int fd = create_file(filename);

    // Structure for flock data structure
    struct flock fl;
    memset(&fl, 0, sizeof(fl));

    fl.l_type = F_WRLCK;  // Exclusive write lock
    fl.l_whence = SEEK_SET;
    fl.l_start = 0;       // Start of the file
    fl.l_len = 0;         // Lock entire file
    fl.l_pid = getpid();

    // Parent process applies the lock
    if (fcntl(fd, F_SETLK, &fl) == -1) {
        perror("Failed to set lock by parent");
        close(fd);
        exit(1);
    }
    printf("Parent: File locked.\n");

    pid_t child_pid = fork();

    if (child_pid == 0) { // Child process
        check_lock(fd, &fl);
        sleep(1);
        check_lock(fd, &fl);
    } else { // Parent process
        sleep(1); // Delay to let child try writing
        printf("Parent: Writing to the file.\n");
        write_file(fd, "Parent writing to file.\n");
       // Release the lock
       release_lock(fd, &fl);
    }

    // Close file descriptor
    close(fd);
    return 0;
}


/*
Note: in this code, modified to return file descriptor after creating file.
Modified write_file, read_file, check_lock, release_lock to accept fd as a param
*/

Overwriting test.c


In [22]:
%%shell
gcc test.c -o test
./test

Hello World
File 'testfile.txt' created.
Parent: File locked.
Child: File is locked by another process (PID: 9898), cannot write.
Parent: Writing to the file.
Child: Lock is free, writing to the file.
